In [1]:
from keras.preprocessing.image import ImageDataGenerator

def  ImportImages (train_path = "" ,test_path = ""):

    train_datagen = ImageDataGenerator(
        rescale=1.0/255 ,
        rotation_range=45 ,
        width_shift_range=0.2 ,
        height_shift_range=0.2 ,
        shear_range=0.2 ,
        zoom_range=0.2 ,
        horizontal_flip=True)
    
    test_datagen = ImageDataGenerator(rescale=1.0/255)

    train_generator = train_datagen.flow_from_directory(
        train_path ,
        target_size=(200 ,200),
        batch_size=32,
        class_mode='binary'
    )
    test_generator = test_datagen.flow_from_directory(
        test_path ,
        target_size=(200 ,200),
        batch_size=32,
        class_mode='binary'
    )

    return (train_generator ,test_generator)

In [2]:
from keras.layers import Dense ,Dropout ,MaxPooling2D ,Conv2D ,Flatten ,BatchNormalization ,add
from keras.layers.experimental.preprocessing import RandomFlip ,RandomRotation ,Rescaling
from keras.models import Model ,Sequential
from keras import Input
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from tensorflow import expand_dims
import kerastuner as kt

In [3]:
train_data ,test_data = ImportImages(train_path='./train' ,test_path='./test')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [4]:
def Xception_layer(hp):
    inputs = Input(shape=(200 ,200 ,3))
    num_block = hp.Int('num_block', min_value=2, max_value=5, step=1)
    num_filters = hp.Int('num_filters', min_value=8, max_value=32, step=8)
    layer = Conv2D(num_filters ,(num_block,num_block) ,activation='relu' ,padding='same')(inputs)
    layer = BatchNormalization()(layer)
    layer = MaxPooling2D((2,2))(layer)
    previous_block_layer = layer
    
    for _ in range(4) :
        num_filters_1 = hp.Int('num_filters_1', min_value=16, max_value=64, step=16)
        layer = Conv2D(num_filters_1 ,(3,3) ,activation='relu' ,padding='same')(layer)
        layer = BatchNormalization()(layer)
        
        num_filters_2 = hp.Int('num_filters_2', min_value=32, max_value=128, step=16)
        layer = Conv2D(num_filters_2 ,(3,3) ,activation='relu' ,padding='same')(layer)
        layer = BatchNormalization()(layer)
        layer = MaxPooling2D((2,2))(layer)
        
        layer2 = Conv2D(num_filters_2 ,(3,3) ,activation='relu' ,padding='same')(previous_block_layer)
        layer2 = BatchNormalization()(layer2)
        layer2 = MaxPooling2D((2,2))(layer2)
        
        layer = add([layer ,layer2])
        previous_block_layer = layer
    
    layer = Flatten()(layer)
    layer = Dropout(hp.Float('dense_dropout', min_value=0., max_value=0.7))(layer)
    layer = Dense(units=hp.Int('num_dense_units', min_value=32, max_value=256, step=32),activation="relu")(layer)
    outputs = Dense(1 ,activation="sigmoid")(layer)
    model = Model(inputs ,outputs)
    
    print(model.summary())
    
    lr = hp.Choice('learning_rate' ,[0.001 ,0.0001 ,0.00001])
    model.compile(loss=binary_crossentropy,
             optimizer=Adam(learning_rate=lr),
             metrics=['acc'])
    
    return model

In [5]:
tuner = kt.BayesianOptimization( Xception_layer, objective = 'val_acc', max_trials=10 ,overwrite=True)

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 200, 200, 8)  104         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 200, 200, 8)  32          conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 100, 100, 8)  0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [6]:
tuner.search(train_data ,validation_data = test_data ,epochs=10 ,steps_per_epoch =200)

Trial 10 Complete [00h 10m 09s]
val_acc: 0.784600019454956

Best val_acc So Far: 0.8270000219345093
Total elapsed time: 01h 45m 38s
INFO:tensorflow:Oracle triggered exit


In [7]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_hps

In [8]:
tuner.results_summary()

Results summary
Results in .\untitled_project
Showing 10 best trials
Objective(name='val_acc', direction='max')
Trial summary
Hyperparameters:
num_block: 2
num_filters: 32
num_filters_1: 16
num_filters_2: 128
dense_dropout: 0.0
num_dense_units: 32
learning_rate: 0.001
Score: 0.8270000219345093
Trial summary
Hyperparameters:
num_block: 2
num_filters: 16
num_filters_1: 32
num_filters_2: 128
dense_dropout: 0.6884907542252953
num_dense_units: 96
learning_rate: 0.001
Score: 0.8001999855041504
Trial summary
Hyperparameters:
num_block: 2
num_filters: 32
num_filters_1: 64
num_filters_2: 128
dense_dropout: 0.0
num_dense_units: 32
learning_rate: 0.001
Score: 0.784600019454956
Trial summary
Hyperparameters:
num_block: 2
num_filters: 8
num_filters_1: 64
num_filters_2: 128
dense_dropout: 0.7
num_dense_units: 256
learning_rate: 0.001
Score: 0.7789999842643738
Trial summary
Hyperparameters:
num_block: 2
num_filters: 32
num_filters_1: 32
num_filters_2: 128
dense_dropout: 0.5802761745839086
num_dense_u

In [9]:
model = tuner.hypermodel.build(best_hps)

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 200, 200, 32) 416         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 200, 200, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 100, 100, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [10]:
model.fit(train_data ,validation_data=test_data ,epochs=100 ,steps_per_epoch=200)

Epoch 1/100
200/200 [==============================] - 60s 298ms/step - loss: 0.7139 - acc: 0.5653 - val_loss: 0.6935 - val_acc: 0.5000
Epoch 2/100
200/200 [==============================] - 59s 297ms/step - loss: 0.6526 - acc: 0.6136 - val_loss: 0.6637 - val_acc: 0.5818
Epoch 3/100
200/200 [==============================] - 59s 297ms/step - loss: 0.6477 - acc: 0.6067 - val_loss: 0.6344 - val_acc: 0.6218
Epoch 4/100
200/200 [==============================] - 59s 297ms/step - loss: 0.6296 - acc: 0.6533 - val_loss: 0.7369 - val_acc: 0.5880
Epoch 5/100
200/200 [==============================] - 60s 298ms/step - loss: 0.6204 - acc: 0.6591 - val_loss: 0.7183 - val_acc: 0.5842
Epoch 6/100
200/200 [==============================] - 60s 298ms/step - loss: 0.6190 - acc: 0.6689 - val_loss: 0.5955 - val_acc: 0.6810
Epoch 7/100
200/200 [==============================] - 60s 298ms/step - loss: 0.5968 - acc: 0.6820 - val_loss: 0.8608 - val_acc: 0.6314
Epoch 8/100
200/200 [===========================

Epoch 61/100
200/200 [==============================] - 60s 299ms/step - loss: 0.1430 - acc: 0.9422 - val_loss: 0.1992 - val_acc: 0.9196
Epoch 62/100
200/200 [==============================] - 60s 301ms/step - loss: 0.1418 - acc: 0.9414 - val_loss: 0.1280 - val_acc: 0.9478
Epoch 63/100
200/200 [==============================] - 60s 300ms/step - loss: 0.1307 - acc: 0.9459 - val_loss: 0.1510 - val_acc: 0.9394
Epoch 64/100
200/200 [==============================] - 60s 300ms/step - loss: 0.1479 - acc: 0.9381 - val_loss: 0.1300 - val_acc: 0.9482
Epoch 65/100
200/200 [==============================] - 60s 301ms/step - loss: 0.1358 - acc: 0.9470 - val_loss: 0.1754 - val_acc: 0.9262
Epoch 66/100
200/200 [==============================] - 60s 300ms/step - loss: 0.1342 - acc: 0.9434 - val_loss: 0.2202 - val_acc: 0.9014
Epoch 67/100
200/200 [==============================] - 60s 299ms/step - loss: 0.1303 - acc: 0.9495 - val_loss: 0.1269 - val_acc: 0.9482
Epoch 68/100
200/200 [===================

In [11]:
model.save("Xcep_tuner.h5")